In [9]:
import cvxpy as cp
import numpy as np
from itertools import product

In [16]:
shipping_cost = np.array([[1,3,2],[3,2,2]]).reshape(2,3)
all_demand = np.array([[100,150,120],[120,180,150],[150,200,180]]).reshape(3,3)
selling_price = np.array([[16,16]]).reshape(2,1)
produce = np.array([200,100]).reshape(2,1)
production_cost = np.array([11,10]).reshape(2,1)

In [17]:
senario = np.array([[100,120,150],
       [150,180,200],[120,150,180]])
senario_comb = []
index = np.arange(0,3)
index_permute = product(index,repeat=3)
for i in index_permute:
    temp_array =[]
    for j in range(0,3):
        temp_array.append(float(senario[j,:][i[j]]))
    senario_comb.append(temp_array)
all_demand = np.array(senario_comb).reshape(27,3)

In [18]:
for demand in all_demand:
    print(demand)

[100. 150. 120.]
[100. 150. 150.]
[100. 150. 180.]
[100. 180. 120.]
[100. 180. 150.]
[100. 180. 180.]
[100. 200. 120.]
[100. 200. 150.]
[100. 200. 180.]
[120. 150. 120.]
[120. 150. 150.]
[120. 150. 180.]
[120. 180. 120.]
[120. 180. 150.]
[120. 180. 180.]
[120. 200. 120.]
[120. 200. 150.]
[120. 200. 180.]
[150. 150. 120.]
[150. 150. 150.]
[150. 150. 180.]
[150. 180. 120.]
[150. 180. 150.]
[150. 180. 180.]
[150. 200. 120.]
[150. 200. 150.]
[150. 200. 180.]


In [19]:
def solve_second_stage_problem(produce,all_demand):

    duals = []
    vs = []

    for demand in all_demand:
        
        sell = cp.Variable(shape=(2,3))
        salvage = cp.Variable(shape=(2,1))
        
        objective = cp.Minimize(-16*cp.sum(sell,keepdims=True) + cp.trace(np.identity((2))@(((shipping_cost@(sell.T))))))
        constraints = [cp.sum(sell,axis=1,keepdims=True) +salvage <= produce,cp.sum(sell,axis=0,keepdims=True) <= demand.reshape(1,-1), sell>= np.zeros_like(sell),salvage>= np.zeros_like(salvage)]

        problem = cp.Problem(objective, constraints)
        problem.solve()

        dual_value = problem.constraints[0].dual_value

        duals.append(dual_value)
        vs.append(problem.value)
        
    return np.array(duals).reshape(-1,2),np.array(vs).reshape(-1,1)

In [20]:
def solve_first_stage_problem(produce,limit,g_ks,alpha_ks): 

    size = 2
    production_cost = np.array([11,10]).reshape(2,1)

    produce  = cp.Variable(shape=(2,1))
    v = cp.Variable(shape=(1,1))
    produce_limit = np.array([300,300]).reshape(2,1)

    constraints = []
    for i in range(0,len(g_ks)):
        constraints.append(np.array(g_ks[i]).T@produce+np.array(alpha_ks[i])<=v)

    constraints.extend([produce>=0,v>=limit,produce<=produce_limit])

    objective = cp.Minimize(production_cost.T@produce + v)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return produce.value, v.value

In [21]:
# produce = np.array([3,3,3]).reshape(-1,1)
limit = -100000

g_ks = []
alpha_ks = []

for i in range(0,300):
    duals,qs = solve_second_stage_problem(produce,all_demand)

    print(duals)

    senario = 27
    senario_prob = (1/senario)*(np.ones_like(all_demand[:,0])).reshape(-1,1)
    g_ks_temp = (-senario_prob.T@duals).T
    alpha_ks_temp = senario_prob.T@qs - g_ks_temp.T@produce

    # print(senario_prob.T@qs)

    g_ks.append(g_ks_temp)
    alpha_ks.append(alpha_ks_temp)

    produce,limit= solve_first_stage_problem(produce,limit,g_ks,alpha_ks)
    print(produce)
    
    # print(limit)

[[14.         14.        ]
 [14.00000001 14.00000001]
 [14.00000001 14.00000001]
 [14.         14.        ]
 [14.00000002 14.00000002]
 [14.00000001 14.00000001]
 [14.         14.        ]
 [14.00000002 14.00000002]
 [14.00000001 14.00000001]
 [14.00000001 14.00000002]
 [14.00000002 14.00000001]
 [14.         14.        ]
 [13.99999996 14.00000002]
 [14.00000002 14.00000001]
 [14.         14.        ]
 [13.99999995 14.00000002]
 [14.00000001 14.00000001]
 [14.         14.        ]
 [14.00000002 14.00000002]
 [14.00000001 14.        ]
 [14.00000001 14.        ]
 [14.00000002 14.00000002]
 [14.         14.        ]
 [14.00000001 14.        ]
 [14.00000001 14.00000001]
 [14.         14.        ]
 [14.00000001 14.        ]]
[[299.9999996 ]
 [299.99999985]]
[[4.99053651e-08 1.05604994e-07]
 [1.02994675e-09 1.61658033e-09]
 [1.49576111e-09 2.06394028e-09]
 [2.94637793e-09 4.25507815e-09]
 [2.98656891e-09 4.15180263e-09]
 [2.01240004e-09 2.94945929e-09]
 [6.07989759e-09 7.79596226e-09]
 [4.55

KeyboardInterrupt: 